In [4]:
# initialize environment
from urbantrips.datamodel.misc import create_line_and_branches_metadata
from urbantrips.utils import utils

# Check config file consistency
utils.check_config()

# Create basic dir structure:
utils.create_directories()

# Create DB:
utils.create_db()
create_line_and_branches_metadata()

Chequeando archivo de configuracion
Proceso de chequeo de archivo de configuración concluido con éxito
create_db Bases abiertas con exito
Tablas originales creadas
Fin crear base
 Finalizado. Tardo 1.50 segundos
create_line_and_branches_metadata No hay tabla con informacion configs
 Finalizado. Tardo 0.01 segundos


In [5]:
# process_transactions
from urbantrips.datamodel import legs, trips
from urbantrips.datamodel import transactions as trx
from urbantrips.destinations import destinations as dest
from urbantrips.geo import geo
from urbantrips.carto import carto
from urbantrips.utils import utils



# Check config file consistency
utils.check_config()

# Read config file
configs = utils.leer_configs_generales()
geolocalizar_trx_config = configs["geolocalizar_trx"]

# trx configs
nombres_variables_trx = configs["nombres_variables_trx"]
formato_fecha = configs["formato_fecha"]
col_hora = configs["columna_hora"]
tipo_trx_invalidas = configs["tipo_trx_invalidas"]
nombre_archivo_trx = configs["nombre_archivo_trx"]

tolerancia_parada_destino = configs["tolerancia_parada_destino"]
resolucion_h3 = configs["resolucion_h3"]
trx_order_params = {
    "criterio": configs["ordenamiento_transacciones"],
    "ventana_viajes": configs["ventana_viajes"],
    "ventana_duplicado": configs["ventana_duplicado"],
}

# gps configs
if geolocalizar_trx_config:
    nombre_archivo_gps = configs["nombre_archivo_gps"]
    nombres_variables_gps = configs["nombres_variables_gps"]
else:
    nombre_archivo_gps = None
    nombres_variables_gps = None

# Compute tolerance in h3 ring
ring_size = geo.get_h3_buffer_ring_size(
    resolucion_h3, tolerancia_parada_destino
)

# Produce transaction table
trx.create_transactions(geolocalizar_trx_config,
                        nombre_archivo_trx,
                        nombres_variables_trx,
                        formato_fecha,
                        col_hora,
                        tipo_trx_invalidas,
                        nombre_archivo_gps,
                        nombres_variables_gps)

# Turn transactions into legs
legs.create_legs_from_transactions(trx_order_params)

# Update destination validation matrix
carto.update_stations_catchment_area(ring_size=ring_size)

# Infer legs destinations
dest.infer_destinations()

# Fix trips with same OD
trips.rearrange_trip_id_same_od()

# Produce trips and users tables from legs
trips.create_trips_from_legs()

# Upload route geometries
carto.upload_routes_geoms()

# Inferir route geometries based on legs data
carto.infer_routes_geoms(plotear_lineas=False)

Chequeando archivo de configuracion
Proceso de chequeo de archivo de configuración concluido con éxito
Se utilizarán hexágonos con un lado igual a 461 m. 
Para la matriz de validacion se usará un buffer de 3 hexágonos.
Se utilizará para la matriz de validacion una distancia máxima de 3227 m entre el origen de la etapa siguiente y las estaciones de la línea de la etapa a validar
Si desea mayor precisión utilice un número más grande de resolucion h3
create_transactions Estableciendo conexion con la db
Abriendo archivos de configuracion
Utilizando los siguientes modos homologados
{'COL': 'autobus', 'TRE': 'tren', 'SUB': 'metro', None: 'brt'}
Leyendo archivo de transacciones
desde archivo csv de transacciones
Filtrando transacciones invalidas: {'tipo_trx_tren': ['CHECK OUT SIN CHECKIN', 'CHECK OUT']}
filtrar_transacciones_invalidas  Finalizado. Tardo 0.06 segundos
Renombrando columnas: {'id_trx': 'id', 'fecha_trx': 'FECHA', 'id_tarjeta_trx': 'id_tarjeta', 'modo_trx': 'modo', 'hora_trx': 'h

C:\Users\sanap\miniconda3\envs\urbantrips\lib\site-packages\statsmodels\nonparametric\smoothers_lowess.py:227: RuntimeWarning: invalid value encountered in divide
  res, _ = _lowess(y, x, x, np.ones_like(x),


Utilizando EPSG:9265 como proyección de coordenadas en metros
Obteniendo lowess linea: 32
Utilizando EPSG:9265 como proyección de coordenadas en metros
Obteniendo lowess linea: 33
Utilizando EPSG:9265 como proyección de coordenadas en metros
Obteniendo lowess linea: 34
Utilizando EPSG:9265 como proyección de coordenadas en metros
Obteniendo lowess linea: 35
Utilizando EPSG:9265 como proyección de coordenadas en metros
Obteniendo lowess linea: 36
Utilizando EPSG:9265 como proyección de coordenadas en metros
Obteniendo lowess linea: 37
Utilizando EPSG:9265 como proyección de coordenadas en metros
Obteniendo lowess linea: 38
Utilizando EPSG:9265 como proyección de coordenadas en metros
Obteniendo lowess linea: 39
Utilizando EPSG:9265 como proyección de coordenadas en metros
Obteniendo lowess linea: 40
Utilizando EPSG:9265 como proyección de coordenadas en metros
Obteniendo lowess linea: 41
Utilizando EPSG:9265 como proyección de coordenadas en metros
Obteniendo lowess linea: 42
Utilizando

C:\Users\sanap\miniconda3\envs\urbantrips\lib\site-packages\statsmodels\nonparametric\smoothers_lowess.py:227: RuntimeWarning: invalid value encountered in divide
  res, _ = _lowess(y, x, x, np.ones_like(x),


Utilizando EPSG:9265 como proyección de coordenadas en metros
Obteniendo lowess linea: 135
Utilizando EPSG:9265 como proyección de coordenadas en metros
Obteniendo lowess linea: 136
Utilizando EPSG:9265 como proyección de coordenadas en metros
Obteniendo lowess linea: 137
Utilizando EPSG:9265 como proyección de coordenadas en metros
Obteniendo lowess linea: 138
Utilizando EPSG:9265 como proyección de coordenadas en metros
Obteniendo lowess linea: 139
Utilizando EPSG:9265 como proyección de coordenadas en metros
Obteniendo lowess linea: 140
Utilizando EPSG:9265 como proyección de coordenadas en metros
Obteniendo lowess linea: 141
Utilizando EPSG:9265 como proyección de coordenadas en metros
Obteniendo lowess linea: 142
Utilizando EPSG:9265 como proyección de coordenadas en metros
Obteniendo lowess linea: 143
Utilizando EPSG:9265 como proyección de coordenadas en metros
Obteniendo lowess linea: 144
Utilizando EPSG:9265 como proyección de coordenadas en metros
Obteniendo lowess linea: 145

...

Utilizando EPSG:9265 como proyección de coordenadas en metros
Obteniendo lowess linea: 285
Utilizando EPSG:9265 como proyección de coordenadas en metros
Obteniendo lowess linea: 286
Utilizando EPSG:9265 como proyección de coordenadas en metros
Obteniendo lowess linea: 287
Utilizando EPSG:9265 como proyección de coordenadas en metros
Obteniendo lowess linea: 288
Utilizando EPSG:9265 como proyección de coordenadas en metros
Obteniendo lowess linea: 289
Utilizando EPSG:9265 como proyección de coordenadas en metros
Obteniendo lowess linea: 290
Utilizando EPSG:9265 como proyección de coordenadas en metros
Obteniendo lowess linea: 291
Utilizando EPSG:9265 como proyección de coordenadas en metros
Obteniendo lowess linea: 292
Utilizando EPSG:9265 como proyección de coordenadas en metros
Obteniendo lowess linea: 293
Utilizando EPSG:9265 como proyección de coordenadas en metros
Obteniendo lowess linea: 294
Utilizando EPSG:9265 como proyección de coordenadas en metros
Obteniendo lowess linea: 295

C:\Users\sanap\miniconda3\envs\urbantrips\lib\site-packages\statsmodels\nonparametric\smoothers_lowess.py:227: RuntimeWarning: invalid value encountered in divide
  res, _ = _lowess(y, x, x, np.ones_like(x),


Utilizando EPSG:9265 como proyección de coordenadas en metros
Obteniendo lowess linea: 366
Utilizando EPSG:9265 como proyección de coordenadas en metros
Obteniendo lowess linea: 367
Imposible de generar una linea lowess para id_linea =  367
Utilizando EPSG:9265 como proyección de coordenadas en metros
Obteniendo lowess linea: 368
Utilizando EPSG:9265 como proyección de coordenadas en metros
Obteniendo lowess linea: 369
Utilizando EPSG:9265 como proyección de coordenadas en metros
Obteniendo lowess linea: 370


C:\Users\sanap\miniconda3\envs\urbantrips\lib\site-packages\statsmodels\nonparametric\smoothers_lowess.py:227: RuntimeWarning: invalid value encountered in divide
  res, _ = _lowess(y, x, x, np.ones_like(x),


Utilizando EPSG:9265 como proyección de coordenadas en metros
Obteniendo lowess linea: 371
Utilizando EPSG:9265 como proyección de coordenadas en metros
Obteniendo lowess linea: 372
Utilizando EPSG:9265 como proyección de coordenadas en metros
Obteniendo lowess linea: 373
Utilizando EPSG:9265 como proyección de coordenadas en metros
Obteniendo lowess linea: 374
Utilizando EPSG:9265 como proyección de coordenadas en metros
Obteniendo lowess linea: 375
Utilizando EPSG:9265 como proyección de coordenadas en metros
Obteniendo lowess linea: 376
Utilizando EPSG:9265 como proyección de coordenadas en metros
Obteniendo lowess linea: 377
Utilizando EPSG:9265 como proyección de coordenadas en metros
Obteniendo lowess linea: 378
Utilizando EPSG:9265 como proyección de coordenadas en metros
Obteniendo lowess linea: 379
Utilizando EPSG:9265 como proyección de coordenadas en metros
Obteniendo lowess linea: 381
Utilizando EPSG:9265 como proyección de coordenadas en metros
Obteniendo lowess linea: 382

In [ ]:
# run_postprocessing

from urbantrips.datamodel.misc import persist_datamodel_tables
from urbantrips.kpi import kpi
from urbantrips.viz import viz
from urbantrips.carto import carto
from urbantrips.utils import utils

utils.check_config()

# Compute and viz route section load by line
kpi.compute_route_section_load(id_linea=False, rango_hrs=False)
viz.visualize_route_section_load(id_linea=False, rango_hrs=False)

# Create TAZs
carto.create_zones_table()

# Create voronoi TAZs
carto.create_voronoi_zones()

# Create distances table
carto.create_distances_table(use_parallel=True)

# Persist datamodel into csv tables
persist_datamodel_tables()

# Poduce main viz
viz.create_visualizations()

# Compute KPI
kpi.compute_kpi()



Chequeando archivo de configuracion
Proceso de chequeo de archivo de configuración concluido con éxito
compute_route_section_load Eliminando datos de carga por tramo para linea False horas False tipo de dia weekday n_sections  10section meters None

        delete from ocupacion_por_linea_tramo
        where hora_min is NULL
        and hora_max is NULL
        and day_type = 'weekday'
         and n_sections = 10 and section_meters is NULL;
Obteniendo datos de etapas y rutas
Computing section load per route ...
Uploading data to db...
 Finalizado. Tardo 4.62 segundos
No hay datos de carga por tramo para estos parametros.
 id_linea: False  rango_hrs: False  n_sections: 10  section_meters: None  day_type: weekday
create_zones_table Creo zonificación para matrices OD
Graba zonas en sql lite
 Finalizado. Tardo 4.66 segundos
create_voronoi_zones Crea zonas de transporte
Graba zonas en sql lite
Zonificación guardada en resultados\png\amba2_muestra10_Zona_voi_map.png
 Finalizado. Tardo 60.13